In [1]:
import tensorflow as tf
import numpy as np

In [11]:
batch_size = 32
num_keypoints = 128
depth = 256
training = True
num_heads = 1
num_layers = 3

# left image
pR = tf.random.uniform([batch_size, num_keypoints, 2])
dR = tf.random.uniform([batch_size, num_keypoints, depth])

pL = pR
dL = dR
# pL = tf.random.uniform([batch_size, num_keypoints//2, 2])
# dL = tf.random.uniform([batch_size, num_keypoints//2, depth])

In [12]:
from superglue.layers import SuperGlue

In [13]:
model = SuperGlue(depth=depth, num_layers=num_layers, num_heads=num_heads)

In [14]:
fA, fB = model(dR, pR, dL, pL, training=training)

In [16]:
model.summary()

Model: "super_glue_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keypoint_encoder_layer_1 (Ke multiple                  111264    
_________________________________________________________________
attentional_message_passing_ multiple                  659200    
_________________________________________________________________
attentional_message_passing_ multiple                  659200    
_________________________________________________________________
attentional_message_passing_ multiple                  659200    
_________________________________________________________________
attentional_message_passing_ multiple                  659200    
_________________________________________________________________
attentional_message_passing_ multiple                  659200    
_________________________________________________________________
attentional_message_passing_ multiple                 

In [17]:
fA.shape, fB.shape

(TensorShape([32, 128, 256]), TensorShape([32, 128, 256]))

In [39]:
dustbin_var = tf.Variable(3.0)

In [40]:
batch_size, num_keypoints, latent_size = fA.shape.as_list()

In [41]:
Sij = tf.matmul(fA, fB, transpose_b=True)

In [42]:
row = dustbin_var * tf.ones([batch_size, 1, num_keypoints])
col = dustbin_var * tf.ones([batch_size, num_keypoints + 1, 1])
Sij_aug = tf.concat([Sij, row], axis=1)
Sij_aug = tf.concat([Sij_aug, col], axis=2)

In [43]:
Sij_aug[0, -4:, -4:]

<tf.Tensor: id=19401, shape=(4, 4), dtype=float32, numpy=
array([[ 840.576  ,  335.8401 ,  520.214  ,    3.     ],
       [ 335.8401 , 1014.63934,  321.977  ,    3.     ],
       [ 520.214  ,  321.977  ,  966.836  ,    3.     ],
       [   3.     ,    3.     ,    3.     ,    3.     ]], dtype=float32)>

In [47]:
a_vec = np.array([[1.0] * num_keypoints + [num_keypoints]])
b_vec = np.array([[1.0] * num_keypoints + [num_keypoints]])
a_vec = tf.convert_to_tensor(a_vec)
b_vec = tf.convert_to_tensor(b_vec)

In [49]:
b_vec.shape

TensorShape([1, 129])

In [45]:
Sij_aug_final = - Sij_aug

In [46]:
from superglue.losses import py_compute_optimal_transport

In [ ]:
tf.py_function(py_compute_optimal_transport)